In [1]:
import geopandas as gpd
import os
import pandas as pd
import plotly.express as px
import ujson as json
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Load the GeoJSON as a GeoDataFrame
gdf = gpd.read_file("../data/Congressional_Districts.geojson")

# Convert all Timestamp objects to strings
gdf = gdf.map(lambda x: str(x) if isinstance(x, pd.Timestamp) else x)


In [ ]:
df_election = pd.read_csv('../data/fips_df.csv')

# Update '00' districts to '01' in GeoJSON data
gdf['DISTRICT'] = gdf['DISTRICT'].replace('00', '01')
df_election['District'] = df_election['District'].apply(lambda x: str(x).zfill(2))

# Convert the STATEFP20 columns to string in both DataFrames
gdf['STATEFP20'] = gdf['STATEFP20'].astype(str).str.zfill(2)
df_election['STATEFP20'] = df_election['STATEFP20'].astype(str).str.zfill(2)

# Convert STATEFP20 columns to string
gdf['STATEFP20'] = gdf['STATEFP20'].astype(str)
df_election['STATEFP20'] = df_election['STATEFP20'].astype(str)
gdf['DISTRICT'] = gdf['DISTRICT'].astype(str)
df_election['District'] = df_election['District'].astype(str)


# Remove any leading or trailing whitespace
gdf['DISTRICT'] = gdf['DISTRICT'].str.strip()
df_election['District'] = df_election['District'].str.strip()

merged_gdf = gdf.merge(df_election, left_on=['STATEFP20', 'DISTRICT'], right_on=['STATEFP20', 'District'], how='left')


geojson = json.loads(merged_gdf.to_json())

# print(gdf.dtypes)
# print(df_election.dtypes)

# Plot using Plotly
fig = px.choropleth_mapbox(merged_gdf, 
                           geojson=geojson, 
                           locations=merged_gdf.index, 
                           color='Vote Percentage',
                           mapbox_style="carto-positron", 
                           hover_name='Name',
                           hover_data=['Party', 'Raised', 'Spent'],
                           center={"lat": 37.0902, "lon": -95.7129}, 
                           zoom=3,
                           opacity=0.5,
                           labels={'Vote Percentage':'Vote Percentage'})

fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()